# FAKE NEWS DETECTOR

## 0. Importación de librerias

Estas son las librerias necesarias:

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import tensorflow as tf
import re

from nltk.stem.porter import PorterStemmer

from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

from sklearn import svm

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalMaxPooling1D, Embedding
from tensorflow.keras.models import Model

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

import matplotlib.pyplot as plt

import requests
from bs4 import BeautifulSoup

import pickle

## 1. Carga de datos y preprocesado

Importamos todo el dataset: https://www.kaggle.com/competitions/fake-news/data

El tran.csv contendrá los datos para entrenar.

El test.csv y submit.csv son los datos para el testeo y sus respuestas respectivamente.

In [2]:
train = pd.read_csv (r'C:\Users\alvar\Desktop\TFG\Dataset\dataset1\train.csv').fillna('')
test = pd.read_csv (r'C:\Users\alvar\Desktop\TFG\Dataset\dataset1\test.csv').fillna('')
submit = pd.read_csv (r'C:\Users\alvar\Desktop\TFG\Dataset\dataset1\submit.csv').fillna('')

Rellenamos los posibles huecos nulos con "fillna()".

### 1.1 Train

Ejemplos del train.

In [3]:
train.sample(3)

,id,title,author,text,label
3890,3890,Why Are Basketball Games So Squeaky? Consider ...,John Branch,"It is the unofficial soundtrack of basketball,...",0
11229,11229,Giant James May terrorises Kent after being gi...,Guest,"November 17, 2016 \nKent woke today to scenes ...",1
6274,6274,Obama Not Going To Bother Fixing White House T...,Gerry McBride,0 Add Comment \n“IT’S not like I’m getting my ...,1


### 1.2 Test

Ejemplos del test.

In [4]:
test.sample(3)

,id,title,author,text
3014,23814,Watch: Hillary Clinton Can’t Stop Coughing Dur...,Pam Key,Friday during her commencement speech at Welle...
3069,23869,Donald Trump: I Would Let My Son Barron Play F...,Charlie Spiering,President Donald Trump would be willing to let...
1025,21825,"Donald Trump Probably Stanched His Losses, but...",Nate Cohn,The bar was low for Donald J. Trump after the ...


### 1.3 Submit

    0 - True
    1 - False

In [5]:
submit.sample(3)

,id,label
281,21081,1
3347,24147,0
67,20867,1


### 1.4 Concatenar autor y título

Aqui juntamos el autor y el titulo en una misma columna para analizarlos juntos. También eliminamos las columnas que ya no nos sirven como text y author.

In [6]:
train.title = train.title + train.author
test.title = test.title + test.author
train.drop(columns=['text', 'author'], inplace=True)
test.drop(columns=['text', 'author'], inplace=True)

### 1.5 StopWords

Los Stopwords son palabras comunes que se filtran o eliminan en el procesamiento del lenguaje natural, ya que se consideran irrelevantes para el análisis de texto o la búsqueda de información. Estas palabras incluyen pronombres, preposiciones, conjunciones y otras palabras que se usan con frecuencia en el lenguaje cotidiano, pero que no aportan información significativa sobre el tema o contenido de un texto.

In [7]:
stopWords = {" ", "a", "ab", "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", 
             "affected", "affecting", "affects", "after", "afterwards", "ag", "again", "against", "ah", "ain", "ain't", "aj", 
             "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", 
             "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", 
             "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", 
             "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't", "arise", "around", "as", 
             "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available", "aw", "away", "awfully", 
             "ax", "ay", "az", "b", "ba", "back", "bc", "bd", "be", "became", "because", "become", 
             "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", 
             "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", 
             "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", 
             "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause", "causes", "cc", "cd", 
             "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", 
             "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", 
             "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", 
             "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc", "dd", "de", 
             "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different", 
             "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", 
             "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "ea", "each", "ec", "ed", "edu", "ee", 
             "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", 
             "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", 
             "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", 
             "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", 
             "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", 
             "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", 
             "fu", "full", "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", 
             "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", 
             "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt", "hasn't", 
             "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", 
             "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", 
             "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit", "however", "how's", 
             "hr", "hs", "http", "hu", "hundred", "hy", "i", "ia", "ib", "ibid", "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", 
             "importance", "important", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", 
             "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward", "io", "ip", "iq", 
             "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", 
             "iz", "j", "jj", "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", 
             "known", "knows", "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", 
             "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely", "line", 
             "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", 
             "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", 
             "merely", "mg", "might", "mightn", "mightn't", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", 
             "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't", 
             "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", 
             "necessary", "need", "needn", "needn't", "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", 
             "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", 
             "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", 
             "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", 
             "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", 
             "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "ow", 
             "owing", "own", "ox", "oz", "p", "page", "pagecount", "pages", "par", "part", "particular", 
             "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", 
             "please", "plus", "pm", "pn", "po", "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", 
             "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", 
             "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", 
             "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", 
             "regardless", "regards", "related", "relatively", "research", "research-articl", "respectively", "resulted", 
             "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", 
             "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", 
             "second", "secondly", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", 
             "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf", "shall", "shan", "shan't", 
             "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", 
             "showed", "shown", "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", 
             "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somebody", "somehow", 
             "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", 
             "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", 
             "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", 
             "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", 
             "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs",
             "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", 
             "there'll", "thereof", "therere", "theres", "there's", "thereto", "thereupon", "there've", "these", "they", 
             "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this", 
             "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", 
             "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", 
             "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "t's", "tt", 
             "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", 
             "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", 
             "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "ut", "v", "va", "value", "various", 
             "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", 
             "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", 
             "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", 
             "what's", "when", "whence", "whenever", "when's", "where", "whereafter", "whereas", "whereby", "wherein", 
             "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", 
             "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", 
             "will", "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", 
             "world", "would", "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", 
             "xn", "xo", "xs", "xt", "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", 
             "your", "youre", "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz"}

### 1.6 Funcion filtro

Vamos a procesar los datos de entrada.

Primero definimos el Stemming:
    
Stemming es el proceso mediante el cual se reduce una palabra a su raiz.

In [8]:
ps = PorterStemmer()

Ahora necesitaremos la funcion filtro:

Quitaremos la raiz de las palabras, stopWords, reduciremos a palabras sin caracteres extraños, todo en minúscula. Entre otras cosas.

In [9]:
def filtro(content):
    contenido = re.sub('[^a-zA-Z]',' ',content)
    contenido = contenido.lower()
    contenido = contenido.split()
    contenido = [ps.stem(word) for word in contenido if word not in stopWords]
    contenido = list(map(ps.stem, contenido))
    contenido = ' '.join(contenido)
    return contenido

### 1.7 Procesamiento con el filtro

Filtramos tanto los datos de entrenamiento como los de testeo.

In [10]:
train.title = train.title.apply(filtro)

In [11]:
train.head(10)

,id,title,label
0,0,hou dem aid comey letter jason chaffetz tweet ...,1
1,1,flynn hillari clinton big woman campu breitbar...,0
2,2,truth firedconsortiumnew,1
3,3,civilian kill singl airstrik identifiedjessica...,1
4,4,iranian woman jail fiction unpublish stori wom...,1
5,5,jacki mason hollywood love trump bomb north ko...,0
6,6,life life luxuri elton john favorit shark pict...,1
7,7,beno hamon win french socialist parti presiden...,0
8,8,excerpt draft script donald trump ampa black c...,0
9,9,channel plan ukrain russia courtesi trump asso...,0


In [12]:
test.title = test.title.apply(filtro)

In [13]:
test.head(10)

,id,title
0,20800,specter trump loosen tongu pur string silicon ...
1,20801,russian warship readi strike terrorist aleppo
2,20802,nodapl nativ american leader vow stay winter f...
3,20803,tim tebow attempt comeback time baseb york tim...
4,20804,keiser report meme war truth broadcast network
5,20805,trump usa antiqu hero clinton presid
6,20806,pelosi call fbi investig russian donald trump ...
7,20807,weekli featur profil randi shannontrevor loudon
8,20808,urban popul boom climat chang wor
9,20809,cognit dissid


### 1.8 Vectorizacion

La vectorización con TF-IDF es un proceso que convierte el texto en un conjunto de números, que pueden ser utilizados para analizar o buscar información en el texto. Esta técnica utiliza una fórmula para asignar valores numéricos a cada palabra en un documento, basándose en la frecuencia con que aparece en el documento y en el corpus completo de documentos. Los documentos que contienen palabras similares tendrán vectores numéricos similares, lo que facilita la comparación y el análisis de documentos mediante técnicas de aprendizaje automático y minería de datos.

In [14]:
vectorizer = TfidfVectorizer()

In [15]:
trainVec = vectorizer.fit_transform(train.title.values)

In [16]:
testVec = vectorizer.transform(test.title.values)

In [17]:
print(trainVec.shape)
print(testVec.shape)

(20800, 23135)
(5200, 23135)


Vamos a bajar el número de componentes de los 23050 hasta una dimensionalidad de 10.

10 es un hiperparametro de dimensionalidad muy competitivo para este problema.

In [18]:
dimensionalidad = 5
tsvd = TruncatedSVD(n_components=dimensionalidad)

trainVec = tsvd.fit_transform(trainVec)
testVec = tsvd.transform(testVec)

In [19]:
print(trainVec.shape)
print(testVec.shape)

(20800, 5)
(5200, 5)


Como se puede apreciar la dimesionalidad del train y del test han sido reducidas hasta 10.

## 2. Entrenamiento de modelos

Separación y entrenamiento.

Dividiremos el fichero train en dos subconjuntos. Uno del 80% y otro del 20%.

In [20]:
X_train, X_test, Y_train, Y_test = train_test_split(trainVec, train.label, test_size=0.2, stratify=train.label, random_state=20)

Se van a usar cuatro modelos distintos para realizar este problema. Mas tarde se compararán sus resultados.

- Modelo SVC no lineal
- Modelo no lineal red neuronal simple
- Modelo no lineal red neuronal compleja (Deep learning)
- Modelo no lineal red neuronal transformer

Para cada modelo tenemos que obtener los hiperparámetros óptimos.

Necesitamos una función para hacer que los resultados de las redes neuronales sean binarios ya que necesitamos clasificar entre verdaderos y falsos.

In [21]:
def binarizador(p, pAux, x):
    for i in p:
        if i >= x:
            pAux.append(1)
        else: 
            pAux.append(0)

### 2.1 Modelo SVC no lineal

SVC: El "Support Vector Classifier" es un clasificador binario. Este funciona con un algoritmo de aprendizaje automático supervisado utilizado para clasificar datos en dos o más categorías.

#### 2.1.1 Selección de hiperparámetros

Los hiperparametros que tendremos que tener en cuenta serán el kernel del algoritmo el cual lo dejaremos en polinomico con su degree a 4. Con el parámetro de regularización a 0.5.

#### 2.1.2 Entrenamiento del modelo

In [22]:
SVC = svm.SVC(kernel="poly", C=0.5, degree=4)
SVC.fit(X_train, Y_train)

SVC(C=0.5, degree=4, kernel='poly')

In [23]:
train_prediction1 = SVC.predict(X_test)

In [24]:
print("Accuracy: ",accuracy_score(train_prediction1, Y_test))
print("Precision: ",precision_score(train_prediction1, Y_test))
print("Recall: ",recall_score(train_prediction1, Y_test))

Accuracy:  0.8079326923076923
Precision:  0.9942390782525204
Recall:  0.7246326102169349


### 2.2 Modelo no lineal: Red neuronal simple

Red neuronal simple: Una red neuronal simple puede utilizarse para clasificar noticias como verdaderas o falsas basándose en características específicas. Sin embargo, es probable que no tenga la capacidad suficiente para identificar patrones muy complejos y sutiles.

#### 2.2.1 Selección de hiperparámetros

Usaremos solo una capa ya que es una red neuronal simple.

En este caso los hiperparámetros son el tipo de conexion entre la capa única y la de salida, el número de neuronas en la capa única y el número de épocas.

Los hiperparametros seleccionados serán los siguientes:


- Unidades en la capa densa (5 y 1): Esta es la cantidad de neuronas en la capa oculta y la capa de salida, respectivamente. En este caso, la capa oculta tiene 5 neuronas y la capa de salida tiene 1 neurona.


- Función de activación ('sigmoid'): Esta es la función de activación utilizada en la capa de salida. La función de activación 'sigmoid' convierte los valores de entrada a un rango entre 0 y 1, lo que la hace adecuada para problemas de clasificación binaria.


- Optimizador ('Adam') y tasa de aprendizaje (0.1): El optimizador es el algoritmo utilizado para ajustar los pesos de la red durante el entrenamiento con el fin de minimizar la función de pérdida. La tasa de aprendizaje es un hiperparámetro que determina cuánto cambian los pesos del modelo en respuesta a los datos de entrenamiento. En este caso, se utiliza una tasa de aprendizaje de 0.1, que es relativamente alta.


- Función de pérdida ('binary_crossentropy'): La función de pérdida mide qué tan bien el modelo está realizando su tarea. Para un problema de clasificación binaria como este, se suele utilizar la entropía cruzada binaria.


- Métrica ('accuracy'): Las métricas son utilizadas para evaluar el rendimiento del modelo. En este caso, se utiliza la precisión, que mide la proporción de predicciones correctas.


- Número de épocas (10): Las épocas son el número de veces que el algoritmo de aprendizaje funcionará en todo el conjunto de datos de entrenamiento. En este caso, se utilizarán 10 épocas.

#### 2.2.2 Entrenamiento del modelo

In [25]:
capaUnica = tf.keras.layers.Dense(units=5)
salidaRNS = tf.keras.layers.Dense(units=1, activation = 'sigmoid')
RNS = tf.keras.Sequential([capaUnica, salidaRNS])

In [26]:
RNS.compile(
    optimizer = tf.keras.optimizers.Adam(0.1),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [27]:
RNS.fit(X_train, Y_train, epochs=10, verbose=False)

In [28]:
train_prediction2 = RNS.predict(X_test)

130/130 [==============================] - 0s 1ms/step


In [29]:
train_prediction2Aux = []
binarizador(train_prediction2, train_prediction2Aux, 0.5) 

In [30]:
print("Accuracy: ",accuracy_score(train_prediction2Aux, Y_test))
print("Precision: ",precision_score(train_prediction2Aux, Y_test))
print("Recall: ",recall_score(train_prediction2Aux, Y_test))

Accuracy:  0.803125
Precision:  0.9054248679788767
Recall:  0.751993620414673


### 2.3 Modelo no lineal: Red neuronal compleja (Deep learning)

Red neuronal compleja (Deep learning): Una red neuronal compleja, también conocida como deep learning, es una evolución de las redes neuronales simples. Puede procesar una gran cantidad de datos y aprender características con cierta complejidad y sutileza que pueden indicar una noticia falsa.

#### 2.3.1 Selección de hiperparámetros

Usaremos varias capas ya que es una red neuronal compleja también conocido como Deep Learning.

En este caso los hiperparámetros son el número de capas, el tipo de conexion entre ellas, el número de neuronas en cada capa y el número de épocas.

Los hiperparametros seleccionados serán los siguientes:


- Unidades en las capas densas (1, 2, 1, 2, 1): Estos son los números de neuronas en cada una de las capas de la red neuronal. En este caso, las capas ocultas tienen 1, 2, 1, y 2 neuronas, respectivamente, y la capa de salida tiene 1 neurona.


- Optimizador ('Adam') y tasa de aprendizaje (0.1): El optimizador es el algoritmo que se utiliza para ajustar los pesos de la red durante el entrenamiento para minimizar la función de pérdida. En este caso se utiliza el optimizador Adam con una tasa de aprendizaje de 0.1. La tasa de aprendizaje es un hiperparámetro que determina cuánto se modifican los pesos del modelo en respuesta a los datos de entrenamiento.


- Función de pérdida ('mean_squared_error'): La función de pérdida es lo que el modelo intenta minimizar durante el entrenamiento. En este caso se usa el error cuadrático medio (MSE).


- Métrica ('accuracy'): Las métricas son usadas para evaluar el rendimiento del modelo. En este caso, se está utilizando la precisión (accuracy), que mide la proporción de predicciones correctas. 


- Número de épocas (20): Las épocas son el número de veces que el algoritmo de aprendizaje trabajará en todo el conjunto de datos de entrenamiento. En este caso, se utilizarán 20 épocas.

#### 2.3.2 Entrenamiento del modelo

In [31]:
capaOc1 = tf.keras.layers.Dense(units=1)
capaOc2 = tf.keras.layers.Dense(units=2)
capaOc3 = tf.keras.layers.Dense(units=1)
capaOc4 = tf.keras.layers.Dense(units=2)
salidaRNC = tf.keras.layers.Dense(units=1)
RNC = tf.keras.Sequential([capaOc1, capaOc2, capaOc3, capaOc4, salidaRNC])

In [32]:
RNC.compile(
    optimizer = tf.keras.optimizers.Adam(0.1),
    loss='mean_squared_error',
    metrics=['accuracy']
)

In [33]:
RNC.fit(X_train, Y_train, epochs=20, verbose=False)

In [34]:
train_prediction3 = RNC.predict(X_test)

130/130 [==============================] - 0s 1ms/step


In [35]:
train_prediction3Aux = []
binarizador(train_prediction3, train_prediction3Aux, 0.5) 

In [36]:
print("Accuracy: ",accuracy_score(train_prediction3Aux, Y_test))
print("Precision: ",precision_score(train_prediction3Aux, Y_test))
print("Recall: ",recall_score(train_prediction3Aux, Y_test))

Accuracy:  0.7978365384615385
Precision:  0.9884781565050408
Recall:  0.7159248956884562


### 2.4 Modelo no lineal: Red neuronal transformer

Red neuronal transformer: Una red neuronal transformer puede procesar secuencias de texto y aprender patrones en el lenguaje que puedan indicar una noticia falsa. Puede ser entrenada para identificar el lenguaje engañoso y las tácticas comunes utilizadas en las noticias falsas.

#### 2.4.1 Selección de hiperparámetros

Los hiperparametros utilizados serán los siguientes:


- Tamaño de entrada (input_shape): Esta es la dimensionalidad de los datos de entrada. Es un hiperparámetro que debe ser igual a la cantidad de características en los datos de entrada.


- Unidades en las capas densas (256, 128, 64, 32, 16, 1): Este es el número de neuronas en cada capa. Es un hiperparámetro que puedes ajustar para mejorar el rendimiento del modelo. Generalmente, un mayor número de neuronas puede capturar más complejidad, pero también puede causar sobreajuste.


- Funciones de activación ('relu', 'sigmoid'): Las funciones de activación determinan la salida de una neurona dada su entrada. 'Relu' (Rectified Linear Unit) es una función de activación común en las capas ocultas, mientras que 'Sigmoid' se utiliza a menudo en la capa de salida para tareas de clasificación binaria.


- Tasa de dropout (0.2): El Dropout es una técnica de regularización que ayuda a prevenir el sobreajuste. Durante el entrenamiento, algunas neuronas de la red (en este caso, el 20% en cada capa) son ignoradas o "apagadas" al azar. Esto obliga a la red a aprender representaciones más robustas de los datos.


- Número de capas (5 capas ocultas): Este es el número de capas en la red, excluyendo las capas de entrada y salida. Aumentar el número de capas puede hacer que el modelo sea más profundo y capaz de aprender representaciones más complejas, pero también puede llevar a sobreajuste y requiere más datos de entrenamiento.


- Tipo de modelo (Dense, Dropout, Input, Model): Este es el tipo de cada capa en la red. 'Dense' es una capa completamente conectada, 'Dropout' es una capa de regularización, 'Input' es la capa de entrada, y 'Model' es la capa final que combina todo en un modelo Keras.

#### 2.4.2 Entrenamiento del modelo

In [37]:
input_layer = Input(shape=(dimensionalidad))
x = Dense(256, activation='relu')(input_layer)
x = Dropout(0.2)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(16, activation='relu')(x)
output_layer = Dense(1, activation='sigmoid')(x)

RNT = Model(inputs=input_layer, outputs=output_layer)

In [38]:
RNT.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

In [39]:
RNT.fit(X_train, Y_train, batch_size=32, epochs=10, verbose=False)

In [40]:
train_prediction4 = RNT.predict(X_test)

130/130 [==============================] - 0s 1ms/step


In [41]:
train_prediction4Aux = []
binarizador(train_prediction4, train_prediction4Aux, 0.5) 

In [42]:
print("Accuracy: ",accuracy_score(train_prediction4Aux, Y_test))
print("Precision: ",precision_score(train_prediction4Aux, Y_test))
print("Recall: ",recall_score(train_prediction4Aux, Y_test))

Accuracy:  0.8651442307692307
Precision:  0.9380700912145943
Recall:  0.8189438390611903


## 3. Resultados

Ahora vamos a probar nuestro modelos entrenados en el dataset de test. Esto nos dará un porcentaje de acierto en base a como de eficaces sean los modelos. También se harán comparaciones entre modelos.

### 3.1 Modelo Lineal SVC

In [43]:
test_prediction1 = SVC.predict(testVec)

In [44]:
print("Metricas para evaluar modelo SVC: \n")
print("Accuracy: ",accuracy_score(test_prediction1, submit['label'].values))
print("Precision: ",precision_score(test_prediction1, submit['label'].values))
print("Recall: ",recall_score(test_prediction1, submit['label'].values))
print("F1 Score: ",f1_score(test_prediction1, submit['label'].values))

Metricas para evaluar modelo SVC: 

Accuracy:  0.7507692307692307
Precision:  0.8902481649772807
Recall:  0.7217342023236044
F1 Score:  0.7971830985915493


### 3.2 Modelo no lineal: Red neuronal simple

In [45]:
test_prediction2 = RNS.predict(testVec)

163/163 [==============================] - 0s 1ms/step


In [96]:
test_prediction2Aux = []
binarizador(test_prediction2, test_prediction2Aux, 0.2)        

In [97]:
print("Metricas para evaluar modelo RNS: \n")
print("Accuracy: ",accuracy_score(test_prediction2Aux, submit['label'].values))
print("Precision: ",precision_score(test_prediction2Aux, submit['label'].values))
print("Recall: ",recall_score(test_prediction2Aux, submit['label'].values))
print("F1 Score: ",f1_score(test_prediction2Aux, submit['label'].values))

Metricas para evaluar modelo RNS: 

Accuracy:  0.759423076923077
Precision:  0.9084236281020622
Recall:  0.7243589743589743
F1 Score:  0.806016436656846


### 3.3 Modelo no lineal: Red neuronal compleja (Deep learning)

In [48]:
test_prediction3 = RNC.predict(testVec)

163/163 [==============================] - 0s 2ms/step


In [90]:
test_prediction3Aux = []
binarizador(test_prediction3, test_prediction3Aux, 0.45)

In [91]:
print("Metricas para evaluar modelo RNC: \n")
print("Accuracy: ",accuracy_score(test_prediction3Aux, submit['label'].values))
print("Precision: ",precision_score(test_prediction3Aux, submit['label'].values))
print("Recall: ",recall_score(test_prediction3Aux, submit['label'].values))
print("F1 Score: ",f1_score(test_prediction3Aux, submit['label'].values))

Metricas para evaluar modelo RNC: 

Accuracy:  0.7596153846153846
Precision:  0.9108703250611674
Recall:  0.7236878644820883
F1 Score:  0.8065614360878984


### 3.4 Modelo no lineal: Red neuronal transformer

In [51]:
test_prediction4 = RNT.predict(testVec)

163/163 [==============================] - 0s 2ms/step


In [75]:
test_prediction4Aux = []
binarizador(test_prediction4, test_prediction4Aux, 0.05)

In [76]:
print("Metricas para evaluar modelo RNT: \n")
print("Accuracy: ",accuracy_score(test_prediction4Aux, submit['label'].values))
print("Precision: ",precision_score(test_prediction4Aux, submit['label'].values))
print("Recall: ",recall_score(test_prediction4Aux, submit['label'].values))
print("F1 Score: ",f1_score(test_prediction4Aux, submit['label'].values))

Metricas para evaluar modelo RNT: 

Accuracy:  0.7623076923076924
Precision:  0.9150646627053478
Recall:  0.7250069232899474
F1 Score:  0.8090234857849197


### 3.5 Comparación de modelos

Como podemos ver la precisión de los modelos van mejorando ligeramente a medida que son mas complejos.

In [98]:
print("SVC: ",accuracy_score(test_prediction1, submit['label'].values))
print("RNS: ",accuracy_score(test_prediction2Aux, submit['label'].values))
print("RNC: ",accuracy_score(test_prediction3Aux, submit['label'].values))
print("RNT: ",accuracy_score(test_prediction4Aux, submit['label'].values))

SVC:  0.7507692307692307
RNS:  0.759423076923077
RNC:  0.7596153846153846
RNT:  0.7623076923076924


### 3.6 Union de modelos

Este implementacion de unir el resultado de los 4 modelos se ha hecho con el objetivo de intentar solventar la carencias de cada uno.

In [78]:
predictionSuma = []
for i in range(len(submit['label'])):
    suma = test_prediction1[i] + test_prediction2Aux[i] + test_prediction3Aux[i] + test_prediction4Aux[i]
    if(suma == 0 or suma == 1):
        predictionSuma.append(0)
    elif(suma == 3 or suma == 4):
        predictionSuma.append(1)
    else:
        predictionSuma.append(test_prediction4Aux[i])

In [79]:
print("Union de modelos: ",accuracy_score(predictionSuma, submit['label'].values))

Union de modelos:  0.7625


Se ha aumentado la precision pero de forma muy poco significativa.

## 4. Evaluación externa

En esta parte utilizamos un enlace de una noticia externa la dataset para tratar de obtener la información necesaria de la misma. Para después procesarla como hariamos con otra noticia cualquiera del dataset.

El siguiente método, mediante la utilización de la libreria BeautifulSoup, es el que extrae los atributos necesarios del enlace:

In [307]:
def getInfo(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    titulo1 = soup.find('h1')
    titulo2 = soup.find('title')
    titulo = ""
    if titulo1 != None:
        titulo += titulo1.get_text()
    elif titulo2 != None:
        titulo += titulo2.get_text()
    else: 
        titulo = ""

    author1 = soup.find('span', class_='author')
    author2 = soup.find('span', class_='byline__name')
    author3 = soup.find('div', class_='author')
    author4 = soup.find('div', class_='byline__name')
    author_tag = soup.find('meta', attrs={'name': 'author'})
    author5 = author_tag['content'] if author_tag else None

    author = ""
    if author1 != None:
        author += author1.get_text()
    elif author2 != None:
        author += author2.get_text()
    elif author3 != None:
        author += author3.get_text()
    elif author4 != None:
        author += author4.get_text()
    elif author5 != None:
        author += author5    
    else: 
        author = ""    
        
    return titulo + author

In [308]:
enlace = "https://www.breitbart.com/sports/2017/05/23/social-media-happy-shot-elephant-crushed-a-hunter-to-death/amp/"
noticiaExterna = getInfo(enlace)

In [309]:
noticiaExterna = filtro(noticiaExterna)
noticiaExterna = [noticiaExterna]

Esta es la información que queda después de procesarla:

In [310]:
print(noticiaExterna)

['social media happi shot eleph crush hunter death']


Vectorizamos:

In [311]:
noticiaExternaVec = vectorizer.transform(noticiaExterna)

In [312]:
noticiaExternaVec.shape

(1, 23135)

Reducimos su dimensionalidad:

In [313]:
noticiaExternaVec = tsvd.transform(noticiaExternaVec)

In [314]:
noticiaExternaVec.shape

(1, 5)

Finalmente somos capaces de analizar la noticia como si fuese del dataset.

In [315]:
predictionExterna1 = SVC.predict(noticiaExternaVec)
predictionExterna2 = RNS.predict(noticiaExternaVec)
predictionExterna3 = RNC.predict(noticiaExternaVec)
predictionExterna4 = RNT.predict(noticiaExternaVec)

1/1 [==============================] - 0s 17ms/step


In [316]:
print(predictionExterna1)
print(predictionExterna2)
print(predictionExterna3)
print(predictionExterna4)

[1]
[[0.6197667]]
[[0.66725016]]
[[0.61427575]]


## 5. Exportar archivos

Para utilizar en otro proyecto los elementos vectorizer, tsvd y los modelos entrenados sin tener que volver a procesarlos es necesario su exportación a un archivo. Nosotros utilizaremos la libreria pickle y el metodo save de keras tensorflow.

In [317]:
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

In [318]:
with open('tsvd.pkl', 'wb') as f:
    pickle.dump(tsvd, f)

In [319]:
with open('svc.pkl', 'wb') as f:
    pickle.dump(SVC, f)

In [320]:
RNS.save('rns.h5')

In [321]:
RNC.save('rnc.h5')

In [322]:
RNT.save('rnt.h5')

###### End